## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table = 'RPC_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table,
                'filter' : "AND item_sub_department_name = '400 RODENT PEST CONTROL' "})

snowflake_tools - 2025-06-25 19:57:40.931631+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-06-25 19:57:46.480715+10:00 - Saved token with '2025-06-25 23:57:46+10:00' expiry...


## Demographic Segments (AU)

In [6]:
RPC_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [7]:
RPC_ds.head(10)

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 RODENT POISON,Homeowner older kids,482871.29,38847.0,24378,6583657.63,537996.0,325748,929772.57,74269.0,...,460416,0.073344,0.072207,0.074837,0.078366,0.075884,0.075697,0.935919,0.951546,0.988639
1,500 RODENT POISON,Homeowner younger kids,884279.52,73063.0,47627,6583657.63,537996.0,325748,1888397.68,153757.0,...,460416,0.134314,0.135806,0.146208,0.159163,0.157100,0.155136,0.843878,0.864455,0.942450
2,500 RODENT POISON,Older homeowner,3842599.08,308893.0,176581,6583657.63,537996.0,325748,6040871.15,493919.0,...,460416,0.583657,0.574155,0.542079,0.509154,0.504658,0.505430,1.146328,1.137712,1.072511
3,500 RODENT POISON,Renter no kids,302314.85,28228.0,19955,6583657.63,537996.0,325748,755838.65,69596.0,...,460416,0.045919,0.052469,0.061259,0.063706,0.071109,0.074576,0.720799,0.737863,0.821430
4,500 RODENT POISON,Renter with kids,295709.26,27534.0,19196,6583657.63,537996.0,325748,752507.59,68566.0,...,460416,0.044916,0.051179,0.058929,0.063425,0.070057,0.069985,0.708170,0.730534,0.842023
5,500 RODENT POISON,Unclassifiable,64457.95,5175.0,3039,6583657.63,537996.0,325748,102800.83,8433.0,...,460416,0.009791,0.009619,0.009329,0.008665,0.008616,0.008801,1.129961,1.116370,1.059993
6,500 RODENT POISON,Younger homeowner,711425.68,56256.0,34972,6583657.63,537996.0,325748,1394348.24,110181.0,...,460416,0.108059,0.104566,0.107359,0.117522,0.112577,0.110376,0.919479,0.928842,0.972666
7,500 RODENT REPELLENT,Homeowner older kids,61015.46,1656.0,1289,812161.62,22012.0,17091,929772.57,74269.0,...,460416,0.075127,0.075232,0.075420,0.078366,0.075884,0.075697,0.958675,0.991407,0.996341
8,500 RODENT REPELLENT,Homeowner younger kids,124591.69,3323.0,2651,812161.62,22012.0,17091,1888397.68,153757.0,...,460416,0.153408,0.150963,0.155111,0.159163,0.157100,0.155136,0.963838,0.960937,0.999839
9,500 RODENT REPELLENT,Older homeowner,387390.56,10441.0,7929,812161.62,22012.0,17091,6040871.15,493919.0,...,460416,0.476987,0.474332,0.463928,0.509154,0.504658,0.505430,0.936824,0.939909,0.917888


In [8]:
total_summary = RPC_ds.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('SEGMENT_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('SEGMENT_QUANTITY', 'first')
).reset_index()
total_summary

,ITEM_CLASS_NAME,TOTAL_SALES_ALL_SEGMENTS,TOTAL_QUANTITY_ALL_SEGMENTS
0,500 RODENT POISON,482871.29,38847.0
1,500 RODENT REPELLENT,61015.46,1656.0
2,500 RODENT TRAP,385885.82,33766.0


In [9]:
segment_index_summary = RPC_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SALES_INDEX', 'QUANTITY_INDEX', 'CUSTOMER_INDEX']]
segment_index_summary

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 RODENT POISON,Homeowner older kids,0.935919,0.951546,0.988639
1,500 RODENT POISON,Homeowner younger kids,0.843878,0.864455,0.942450
2,500 RODENT POISON,Older homeowner,1.146328,1.137712,1.072511
3,500 RODENT POISON,Renter no kids,0.720799,0.737863,0.821430
4,500 RODENT POISON,Renter with kids,0.708170,0.730534,0.842023
5,500 RODENT POISON,Unclassifiable,1.129961,1.116370,1.059993
6,500 RODENT POISON,Younger homeowner,0.919479,0.928842,0.972666
7,500 RODENT REPELLENT,Homeowner older kids,0.958675,0.991407,0.996341
8,500 RODENT REPELLENT,Homeowner younger kids,0.963838,0.960937,0.999839
9,500 RODENT REPELLENT,Older homeowner,0.936824,0.939909,0.917888


## Commercial Industry Segment (NZ)

In [10]:
# Get data
RPC_is = con.read_sql_file_into_pandas(
    file="sql/5-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': 10, 
        'table_name': table,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [11]:
total_summary = RPC_is.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('TOTAL_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('TOTAL_QUANTITY', 'first')
).reset_index()

In [12]:
segment_index_summary = RPC_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SALES_INDEX', 'QUANTITY_INDEX', 'CUSTOMER_INDEX']]

In [13]:
# Check data
RPC_is.sort_values(by='SALES_INDEX', ascending=False)

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
10,500 RODENT REPELLENT,Accommodation and Food Services,2058.42,43.0,29,25054.95,553.0,338,19887.26,1147.0,...,6630,0.082156,0.077758,0.085799,0.058587,0.057888,0.069683,1.402299,1.343235,1.231276
22,500 RODENT TRAP,Cleaning Services,5323.08,410.0,145,143965.86,11374.0,3765,9737.75,661.0,...,6630,0.036975,0.036047,0.038513,0.028687,0.033360,0.036501,1.288902,1.080541,1.055122
16,500 RODENT REPELLENT,Manufacturing,4567.68,105.0,38,25054.95,553.0,338,55861.11,3148.0,...,6630,0.182306,0.189873,0.112426,0.164564,0.158878,0.090649,1.107816,1.195093,1.240234
24,500 RODENT TRAP,Electrical Services,7591.90,581.0,258,143965.86,11374.0,3765,16216.85,986.0,...,6630,0.052734,0.051081,0.068526,0.047774,0.049763,0.065008,1.103823,1.026498,1.054116
23,500 RODENT TRAP,Commercial Builder,5522.69,388.0,141,143965.86,11374.0,3765,11920.57,664.0,...,6630,0.038361,0.034113,0.037450,0.035117,0.033512,0.035596,1.092369,1.017941,1.052085
25,500 RODENT TRAP,Landscaper and Gardening Services,6224.18,467.0,226,143965.86,11374.0,3765,13773.32,826.0,...,6630,0.043234,0.041059,0.060027,0.040575,0.041688,0.060181,1.065514,0.984908,0.997441
15,500 RODENT REPELLENT,Landscaper and Gardening Services,1076.92,23.0,21,25054.95,553.0,338,13773.32,826.0,...,6630,0.042982,0.041591,0.062130,0.040575,0.041688,0.060181,1.059318,0.997688,1.032386
7,500 RODENT POISON,Professional Computer and Scientific Services,50487.02,2410.0,848,170428.63,7887.0,4243,94981.60,5602.0,...,6630,0.296236,0.305566,0.199859,0.279811,0.282729,0.193514,1.058700,1.080772,1.032788
9,500 RODENT POISON,Retail and Wholesale Trade,30407.46,1433.0,986,170428.63,7887.0,4243,57252.67,3390.0,...,6630,0.178418,0.181691,0.232383,0.168663,0.171091,0.226395,1.057833,1.061957,1.026449
14,500 RODENT REPELLENT,Electrical Services,1260.85,28.0,23,25054.95,553.0,338,16216.85,986.0,...,6630,0.050323,0.050633,0.068047,0.047774,0.049763,0.065008,1.053364,1.017485,1.046748


## Drop item range table

In [14]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table}
    )

# Brand view

In [15]:
overall_brand = con.read_sql_file_into_pandas(
    file="sql/Overall_by_brand.sql",
    formatting={
        "start_date":                 start_date,
        "end_date":                   end_date,
        "sub_dept":                   "'400 RODENT PEST CONTROL'"        
    })

In [16]:
overall_brand

,CUSTOMER_TYPE_CODE,BRAND_NAME,SALES,TOTAL_TRX,TOTAL_UNITS,NUM_ITEMS_PURCHASED,OVERALL_SALES,OVERALL_TRX
0,Consumer,BIG CHEESE,978429.28,40915,51017.0,26,1250863.29,50286
1,Consumer,KIWICARE,695580.01,31730,37769.0,12,940093.19,39491
2,Consumer,CLIX,0.22,1,1.0,1,0.22,1
3,Consumer,TRAPPED!,295159.09,24038,42779.0,9,373345.33,28896
4,Commercial,KIWICARE,244513.18,7761,9740.0,12,940093.19,39491
5,Commercial,NOOSKI,5341.51,337,457.0,3,26334.23,1752
6,Consumer,NO,6960.74,365,367.0,1,8275.91,437
7,Consumer,NOOSKI,20992.72,1415,1747.0,3,26334.23,1752
8,Commercial,GOTCHA,11099.77,1278,2066.0,3,63059.13,7913
9,Commercial,BIG CHEESE,272434.01,9371,12437.0,26,1250863.29,50286
